In [1]:
import copy

def get_points_code(N,r,F):
    points = []
    traces = {} # элементы со следом j
    traces[0] = [a for a in F if a.trace() == 0]
    for B in F:
        if len(points) >= N:
            return points
        if B not in traces[0]:
            c = 2*(B^(r+1)).trace() / B.trace()
            if c not in traces:
                traces[c] = [a for a in F if a.trace() == c]
            for j in traces[c]:
                if len(points) >= N:
                    return points
                points.append([B, j])
    return points

# Автоморфизм кривой
def aut_sigma(xi,i,r):
    return xi * alpha^((r+1)*r^(i-1))

def FGS_encode(N,l,m,sD,F,T2,f):
    n = m*N
    #print('Длина кода n =', n)
    r = F.characteristic()
    points_code = get_points_code(N,r,F)
    #print("Точки кода:", points_code)
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
#     print("L(l*𝑃∞):", L1)    
    k = len(L1)
    #print('Размерность кода k =', k)
    #print('Скорость кода R >=', round(float(k/n),5))
    d = N-l/m
    #print('Минимальное расстояние кода d >=', d)
    t_min = int((d-1)/2)
    g = T2.genus()
    kappa = ceil((N*(m-sD+1) - (k+g-1) + (sD+1)*(g/2-1) + 1) / (sD+1))
    t = (kappa+l) / (m-sD+1)
    t_max = ceil(N-t)
    #print('Число исправляемых ошибок:', t_min, '<= t <=', t_max)
    #print('Исходное сообщение f =', f)
    f_RR = 0
    for f_el, L1_el in zip(f, L1):
        f_RR += f_el*L1_el
    PR = PolynomialRing(F, 2, ['x1','x2'])
    f_RR = PR(f_RR)
    #print('Исходное сообщение как элемент L(l*𝑃∞):', f_RR)
    y = [[0 for i in range(0,N)] for j in range(0,m)]
    for i in range(0,m):
        for j in range(0,N):
            x_1 = points_code[j][0]
            x_2 = points_code[j][1]
            for count_sigma in range(0,i):
                x_1 = aut_sigma(x_1,1,r)
                x_2 = aut_sigma(x_2,2,r)
            y[i][j] = f_RR(x_1, x_2)
    return y

def FGS_decode(N,l,m,sD,F,T2,y):
    r = F.characteristic()
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
    #print('L(l*𝑃∞):', L1)
    k = len(L1)
    g = T2.genus()
    D = ceil((N*(m-sD+1) - (k+g-1) + (sD+1)*(g/2-1) + 1) / (sD+1)) # Степенной параметр
    L2 = (D*P_inf).basis_function_space() # Базис пространства L(D*𝑃∞)
    #print('L(D*𝑃∞):', L2)
    L3 = ((l+D)*P_inf).basis_function_space() # Базис пространства L((l+D)*𝑃∞)
    #print('L((l+D)*𝑃∞):', L3)
    points_code = get_points_code(N,r,F)
    count_lines = N*(m-sD+1)
    count_columns = len(L3)+len(L2)
    A0_A1_matrix = matrix(F, count_lines, count_columns) # Матрица коэффициентов
    PR = PolynomialRing(F, 2, ['x1','x2'])
    for j in range(0,N):
        x_1 = points_code[j][0]
        x_2 = points_code[j][1]
        for count_sigma in range(0,m-sD+1):
            equation_coefficients = []
            for i in range(0,len(L3)):
                L3_i = PR(L3[i])
                equation_coefficients.append(L3_i(x_1, x_2))
            for i in range(0,len(L2)):
                L2_i = PR(L2[i])
                equation_coefficients.append(L2_i(x_1, x_2)*y[count_sigma][j])
            A0_A1_matrix[j*(m-sD+1)+count_sigma] = equation_coefficients
            x_1 = aut_sigma(x_1,1,r)
            x_2 = aut_sigma(x_2,2,r)
#     print("Матрица для нахождения A0 и A1:")
    #print(A0_A1_matrix)
    recovered_message_list = []
    for x in A0_A1_matrix.right_kernel().basis():
        #print("Частное решение:",x) 
        _.<Y> = PolynomialRing(F)
        A0_coeff = x[:len(L3)]
        A1_coeff = x[len(L3):]
        A0 = 0
        A1 = 0
        for coeff, L3_elem in zip(A0_coeff, L3):
            A0 += coeff*L3_elem
        for coeff, L2_elem in zip(A1_coeff, L2):
            A1 += coeff*L2_elem
        Q = A0 + A1*Y
    #    print("A0 =", A0)
    #    print("A1 =", A1)
    #    print("Q =", Q)

        count_lines = N
        count_columns = len(L1)
        f_matrix = matrix(F, count_lines, count_columns)
        b = [0]*count_lines
        A0_PR = PR(A0)
        A1_PR = PR(A1)
        for j in range(0,count_lines):
            x_1 = points_code[j][0]
            x_2 = points_code[j][1]
            equation_coefficients = []
            for i in range(0,count_columns):
                L1_i = PR(L1[i])
                equation_coefficients.append(L1_i(x_1, x_2)*A1_PR(x_1,x_2))
            f_matrix[j] = equation_coefficients
            b[j] = -A0_PR(x_1,x_2)
        b_vector = vector(b)
     #   print("Матрица для нахождения исходного сообщения:")
     #   for i in f_matrix:
     #       print(i)
     #   print("Столбец свободных членов:", b_vector)
        try:
            recovered_message = f_matrix.solve_right(b_vector)
            recovered_message_list.append(list(recovered_message))
        except:
            pass
            print("Декодирование невозможно")
#         print("-------------------------")
    return recovered_message_list

    
    

l = 6 # Множитель дивизора (l*𝑃∞)
r = 3 # Характеристика поля констант F
e = 2 # Ступень башни Г-Ш
m = 2 # 1 <= m <= r-1 (число вычисляемых автоморфизмов для каждой точки при кодировании)
sD = 1 # sD >= 1 (константа декодера)
N = (r-1)*r^e # Кол-во точек кода
# N = 12
F.<alpha> = GF(r^2) # Базовое поле
T1.<x1> = FunctionField(F); _.<Y> = T1[] # Функциональное поле (первая ступень башни Г-Ш)
T2.<x2> = T1.extension(Y^r + Y - (x1^r / (x1^(r-1) + 1))) # Расширение функционального поля (вторая ступень башни Г-Ш)

# f = [1,alpha,alpha^2,0,1] # Сообщение
f = [1,alpha,alpha^2] # Сообщение

# Выбрать рандомное сообщение
# P_inf = T2.places_infinite()[0]
# L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
# k = len(L1)
# f = [list(F)[randint(0, len(list(F))-1)] for i in range(0,k)]

print("Кодирование:")
f_encode = FGS_encode(N,l,m,sD,F,T2,f)
print("Кодовое слово:", f_encode)

#Внесём ошибки
f_encode[0][0] = 2*alpha + 2
# f_encode[1][1] = alpha+1
# f_encode[0][4] = 2*alpha + 1
# f_encode[1][4] = alpha
# f_encode[1][6] = 0
# f_encode[0][6] = 1
# f_encode[0][2] = 2*alpha
# f_encode[1][3] = 2*alpha

print("Кодовое слово с ошибками:", f_encode)
print("------------------------------------------------------")

print("Декодирование:")

recovered_message_list = FGS_decode(N,l,m,sD,F,T2,f_encode)
if (f in recovered_message_list):
    print("Сообщение успешно декодировано!", f)
    print("Список декодированных векторов:", recovered_message_list)
else:
    print("Ошибка", f, recovered_message_list)
    #print(f'Ошибка: {recovered_message_list}')

    
    
    
    
# Для тестирования:

# max_count_error = 3
# for attempt in range(0,500):
#     f = [list(F)[randint(0, len(list(F))-1)] for i in range(0,5)] # Сообщение
#     f_encode = FGS_encode(N,l,r,e,m,s,F,T2,f)
#     #print("Кодовое слово:", f_encode)
#     f_encode_with_errors = copy.deepcopy(f_encode)
#     count_error = 0
#     already_changed = []
#     while count_error < max_count_error:
#         s1 = randint(0,len(f_encode)-1)
#         s2 = randint(0,len(f_encode[s1])-1)
#         random_index_F = randint(0, len(list(F))-1)
#         error = list(F)[random_index_F]
#         if (error != f_encode_with_errors[s1][s2]) and ([s1,s2] not in already_changed):
#             f_encode_with_errors[s1][s2] = error
#             already_changed.append([s1,s2])
#             count_error += 1
#     recovered_message_list = FGS_decode(N,l,e,r,m,s,F,T2,f_encode_with_errors)
#     if (f in recovered_message_list):
#         continue
#         print("Сообщение успешно декодировано!")
#         #print("Список декодированных векторов:", recovered_message_list)
#     else:
#         print("Ошибка", attempt)
#         #print(f'Ошибка: {recovered_message_list}')


Кодирование:
Кодовое слово: [[0, 0, 0, 2, 2, 2, 2, 2, 2, alpha + 1, alpha + 1, alpha + 1, 1, 1, 1, 2*alpha + 2, 2*alpha + 2, 2*alpha + 2], [alpha + 1, alpha + 1, alpha + 1, 1, 1, 1, 2*alpha + 2, 2*alpha + 2, 2*alpha + 2, 0, 0, 0, 2, 2, 2, 2, 2, 2]]
Кодовое слово с ошибками: [[2*alpha + 2, 0, 0, 2, 2, 2, 2, 2, 2, alpha + 1, alpha + 1, alpha + 1, 1, 1, 1, 2*alpha + 2, 2*alpha + 2, 2*alpha + 2], [alpha + 1, alpha + 1, alpha + 1, 1, 1, 1, 2*alpha + 2, 2*alpha + 2, 2*alpha + 2, 0, 0, 0, 2, 2, 2, 2, 2, 2]]
------------------------------------------------------
Декодирование:
Сообщение успешно декодировано! [1, alpha, alpha + 1]
Список декодированных векторов: [[1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [alpha + 2, alpha, 0], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alpha + 1], [1, alpha, alph

In [2]:
# Код Рида-Соломона

r = 3
q = r^2
K.<x> = GF(r)[]
F.<a> = GF(q, modulus=x^2+x+2)
n, k = 4, 2
evaluation_points = F.list()[1:n+1]
C = codes.GeneralizedReedSolomonCode(evaluation_points, k)
D = codes.decoders.GRSBerlekampWelchDecoder(C)

message = [a,a+1]
print("Исходное сообщение:", message)
codeword = C.encode(vector(F, message))
print("Кодовое слово:", codeword)

# Внесение ошибок
space = C.ambient_space()
number_errors = D.decoding_radius()
Chan = channels.StaticErrorRateChannel(space, number_errors)
codeword_with_errors = Chan(codeword)
print("Кодовое слово с ошибками", codeword_with_errors)

decoded_codeword = D.decode_to_code(codeword_with_errors)
print("Декодированное кодовое слово:", decoded_codeword)

# recovered_message = list(D.decode_to_message(y)) # То же самое, сразу вернёт сообщение
recovered_message = list(D.connected_encoder().unencode(decoded_codeword))
print("Восстановленное сообщение", recovered_message)

Исходное сообщение: [a, a + 1]
Кодовое слово: (a + 1, 2*a, 1, 2)
Кодовое слово с ошибками (2, 2*a, 1, 2)
Декодированное кодовое слово: (a + 1, 2*a, 1, 2)
Восстановленное сообщение [a, a + 1]


In [71]:
import math
import itertools

# Декодер для (F_p)^n
def BaseFieldDecoder(y,e0):
    y = vector(y)
    distance = int(e0)
    delta_list = [i for i in range(-distance, distance+1)] # Диапазон значений для построение списка delta_vector'ов ([-2,-1,0,1,2])

    code_words_list = []
    for delta in itertools.product(delta_list, repeat=len(y)): # Все перестановки delta_vector'ов
        delta_vector = vector(delta)
        v = y + delta_vector
        if ((y-v).norm()) <= e0:
            code_words_list.append(v)

    return code_words_list

# Декодер кода Рида-Соломона
def RSDecoder(n,k,F,codeword_with_errors):
    evaluation_points = F.list()[1:n+1]
    C = codes.GeneralizedReedSolomonCode(evaluation_points, k)
    D = codes.decoders.GRSBerlekampWelchDecoder(C)
    y = vector(F, codeword_with_errors)
    try:
        decoded_codeword = D.decode_to_code(y)
        decoded_codeword_int = list(map(int, decoded_codeword))
        return [decoded_codeword_int]
    except:
        return []
    
# Декодирование решётки конструкции D
def LatticeDecoder(y,i,e0,p):
    ei = e2 = p^i*e0
    w = [round(j) % p for j in y]
    if (i == 0):
        L = BaseFieldDecoder(w, e0)
    else:
        L = RSDecoder(n,k,F,w)
    R_c = []
    for c in L:
        if i == 0:
            res = [b + (round((a-b) / p) * p) for a,b in zip(y, c)]
            R_c.append(res)
        else:
            y_new = [(a-b) / p for a,b in zip(y, c)]
            for v in LatticeDecoder(y_new,i-1,e0,p):   
                rez = [a + p*b for a,b in zip(c,v)]
                if ((vector(y)-vector(rez)).norm() <= ei):
                    R_c.append(rez)
    return R_c

p = 5
F = GF(p)
n, k = 4, 2
i = 1
e0 = 2
y = [8.2, 4.4, 5.9, 6.3]

vectors = LatticeDecoder(y,i,e0,p)
print(vectors)

[[3, -1, 0, 6], [3, -1, 5, 1], [3, -1, 5, 6], [3, -1, 5, 11], [3, -1, 10, 6], [3, -1, 10, 11], [3, 4, 0, 1], [3, 4, 0, 6], [3, 4, 0, 11], [3, 4, 5, 1], [3, 4, 5, 6], [3, 4, 5, 11], [3, 4, 10, 1], [3, 4, 10, 6], [3, 4, 10, 11], [3, 9, 0, 6], [3, 9, 5, 1], [3, 9, 5, 6], [3, 9, 5, 11], [3, 9, 10, 1], [3, 9, 10, 6], [3, 9, 10, 11], [8, -1, 0, 1], [8, -1, 0, 6], [8, -1, 0, 11], [8, -1, 5, 1], [8, -1, 5, 6], [8, -1, 5, 11], [8, -1, 10, 1], [8, -1, 10, 6], [8, -1, 10, 11], [8, 4, 0, 1], [8, 4, 0, 6], [8, 4, 0, 11], [8, 4, 5, 1], [8, 4, 5, 6], [8, 4, 5, 11], [8, 4, 5, 16], [8, 4, 10, 1], [8, 4, 10, 6], [8, 4, 10, 11], [8, 4, 15, 6], [8, 9, 0, 1], [8, 9, 0, 6], [8, 9, 0, 11], [8, 9, 5, 1], [8, 9, 5, 6], [8, 9, 5, 11], [8, 9, 10, 1], [8, 9, 10, 6], [8, 9, 10, 11], [8, 14, 5, 6], [13, -1, 0, 6], [13, -1, 5, 1], [13, -1, 5, 6], [13, -1, 5, 11], [13, -1, 10, 1], [13, -1, 10, 6], [13, -1, 10, 11], [13, 4, 0, 1], [13, 4, 0, 6], [13, 4, 0, 11], [13, 4, 5, 1], [13, 4, 5, 6], [13, 4, 5, 11], [13, 4, 10,

In [6]:
# Проверка предложения 4.2 из Vladut

def H(d):
    return -d*log(d,2) - (1-d)*log((1-d),2)

def E(s,d):
    return H(d) - (2*s)/(2^s-1) - log((2^(2*s)/(2^(2*s)-1)), 2)

def Ad(F,N,weight):
    zero_count = N-weight
    len_F_without_zero = len(F)-1
    Comb = Combinations(N,zero_count).cardinality()
    return Comb*(len_F_without_zero)^(weight)

q = 64
F = GF(q)
N = 84
m = 2
N = N*m
a = 100
s = 3
d = N - a
delta = (N-a)/N

logAd = float(log(Ad(F,N,d), 2))
Esd = float(E(s,delta))


print(0, "<", delta ,"<", 1-2^(-2*s))
print("N =", N)
print("Es(d) =", Esd)
print(logAd, ">=", N*Esd)
print(float(1/64*E(3,32/63)))

0 < 17/42 < 63/64
N = 168
Es(d) = 0.09380513090667958
566.033766157422 >= 15.75926199232217
0.001874301770584414


In [161]:
# Проверка предложения 6.2 из Vladut

F.<alpha> = GF(64)
PR.<x1,x2> = PolynomialRing(F)

zeros = 0
errors = 0

C = x2^8 + x2 + (x1^8)/(x1^7 + 1)
for i in list(F):
    for j in list(F):
        try:
            res = C(i,j)
            if res == 0:
                zeros += 1
        except:
            errors +=1
print(zeros)
print(errors)

456
448


In [67]:
def get_points_code(N,r,F):
    points = []
    traces = {} # элементы со следом j
    traces[0] = [a for a in F if a.trace() == 0]
    for B in F:
        if len(points) >= N:
            return points
        if B not in traces[0]:
            c = 2*(B^(r+1)).trace() / B.trace()
            if c not in traces:
                traces[c] = [a for a in F if a.trace() == c]
            for j in traces[c]:
                if len(points) >= N:
                    return points
                points.append([B, j])
    return points

# Функция возвращает порождающую матрицу и размерность подкода
# на ступени T башни Гарсии-Штихтенота для заданного
# множителя дивизора l
# Примечание: вектора матрицы G представлены над (F_2)^n, применена биекция phi.
def gen_code_over_GS_Tower(l,T,N,r,F):
    P_inf = T.places_infinite()[0]
    L = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
#     print('L(l*𝑃∞):', L)
    k = len(L)
    points_code = get_points_code(N,r,F)
    PR = PolynomialRing(F, 2, ['x1','x2'])
    M = [[PR(xi)(Pi) for Pi in points_code] for xi in L]
    G = []
    for v in M:
        new_vector = get_bijection_phi_vector(v, F.degree())
        G.append(new_vector)
    return [G,k]

# Генератор подмассивов длины n из массива lst
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
# Функция представляет элемент конечного поля в виде вектора коэффициентов
# относительно базисных элементов поля
# Пример: element = alpha + alpha^3 + alpha^5 -> [0, 1, 0, 1, 0, 1]
def bijection_phi(element, extension_degree):
    coefficients = list(map(int, element.polynomial().coefficients(sparse=False)))
    return coefficients + [0]*(extension_degree - len(coefficients))

# Функция представляет вектора коэффициентов относительно базисных элементов поля 
# в виде элемента конечного поля  
# Пример: [0, 1, 0, 1, 0, 1] -> element = alpha + alpha^3 + alpha^5
def reverse_bijection_phi(vector, Field):
    generator = Field.gen()
    return reduce(lambda acc, iv: acc + iv[1]*generator^iv[0], enumerate(vector), 0)

# Функция представляет вектор из элементов конечного поля в виде
# конкатинированных результатов применения биекции фи к каждому элементу 
# Пример над F_64: [alpha, alpha^3, alpha^5] -> [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
def get_bijection_phi_vector(vector, extension_degree):
    bijection_phi_vector = []
    for element in vector:
        bijection_phi_vector += bijection_phi(element, extension_degree)
    return bijection_phi_vector

# Обратное преобразование для функции get_bijection_phi_vector
# Пример над F_64: [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1] -> [alpha, alpha^3, alpha^5]
def get_reverse_bijection_phi_vector(vector, Field, extension_degree):
    return list(map(lambda submas: reverse_bijection_phi(submas, Field), chunks(vector, extension_degree)))

# Функция возвращает список векторов, с помощью которых 
# меньшее векторное пространство расширяется до 
# большего векторного пространства
def get_extension_vectors(basis_smaller_vector_space, basis_larger_vector_space, relative_space):
    extension_vector = []
    for vector in basis_larger_vector_space:
        if (vector not in basis_smaller_vector_space):
            if (relative_space.are_linearly_dependent(basis_smaller_vector_space + extension_vector + [vector]) == false):
                extension_vector.append(vector)
    return extension_vector

def gaus(M):
    m = M.nrows()
    for inductive_step in range(m-1,-1,-1):
        for j in range(inductive_step,-1,-1):
            if M[inductive_step][j] == 1:
                M.swap_columns(inductive_step,j)
                break
        for i in range(inductive_step-1,-1,-1):
            if M[i][inductive_step] != 0:
                M.add_multiple_of_row(i,inductive_step,-M[i][inductive_step])
    return M

# Функция фозвращает порождающую матрицу решётки конструкции D
def get_lattice_basis(G0, k_diff, l_nesting):
    L_dim = G0.nrows()
    L = matrix(ZZ, L_dim)
    start_index_row = 0
    for i in range(0, l_nesting+1):
        end_index_row = start_index_row + k_diff[i]
        for index_row in range(start_index_row,end_index_row):
            new_row = list(map(lambda x: int(x)*2^(l_nesting-i), G0[index_row]))
            L[index_row] = new_row
        start_index_row = end_index_row
    return L


r = 2 # Характеристика поля констант F
s = 2
q = r^s
extension_degree_F_2 = s*2
e = 2 # Ступень башни Г-Ш
m = 1 # 1 <= m <= r-1 (число вычисляемых автоморфизмов для каждой точки при кодировании)
sD = 1 # sD >= 1 (константа декодера)
N = (r-1)*r^e # Кол-во точек кода
N = 12
F.<alpha> = GF(r^extension_degree_F_2)
T1.<x1> = FunctionField(F); _.<Y> = T1[] # Функциональное поле (первая ступень башни Г-Ш)
T2.<x2> = T1.extension(Y^q + Y - (x1^q / (x1^(q-1) + 1))) # Расширение функционального поля (вторая ступень башни Г-Ш)
l_nesting = 2 # Число вложенных кодов: C0 \in C1 \in ... \in Cl       
     
    
# Код C2 на второй ступени башни Г-Ш
l2 = 6
[G2, k2] = gen_code_over_GS_Tower(l2,T2,N,r,F)

# Код C1 на второй ступени башни Г-Ш
l1 = 8
[G1, k1] = gen_code_over_GS_Tower(l1,T2,N,r,F)


# Векторные пространства соответствующих кодов
V0 = F^(N*extension_degree_F_2)
V1 = V0.subspace_with_basis(G1)
V2 = V1.subspace_with_basis(G2)

V0_basis = V0.basis()
V1_basis = V1.basis()
V2_basis = V2.basis()
# print(V0_basis)
# print(V1_basis)
# print(V2_basis)
# print("-------------------")


K2 = list(V2_basis)
K1 = get_extension_vectors(V2_basis, V1_basis, V1)
K0 = get_extension_vectors(V1_basis, V0_basis, V0)


print("K0:")
for i in K0:
    print(i)
print("-------------------")
print("K1:")
for i in K1:
    print(i)
print("-------------------")  
print("K2:")
for i in K2:
    print(i)
print("-------------------")

G0 = matrix(F, K0+K1+K2)
print("G0 = K0 + K1 + K2:")
print(G0)
print('----------------')

G0 = gaus(G0)
print("G0 with Gaussian elimination:")
print(G0)
print("-------------------")

# Базисная матрица решётки
k_diff = [len(K0), len(K1), len(K2)]
L = get_lattice_basis(G0, k_diff, l_nesting)
print("Lattice L:")
print(L)

K0:
(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

G0 with Gaussian elimination:
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [68]:
import math
import itertools

# Декодер для (F_2)^n
# Функция возвращает генератор векторов v: ||y-v|| <= e0
def BaseFieldDecoder(y,e0):
    y = vector(y)
    y_len = len(y)
    code_words_list = []
    for k in range(int(e0^2)+1):
        for bits in itertools.combinations(range(y_len), k):
            delta = [0] * y_len
            for bit in bits:
                delta[bit] = 1
            delta_vector = vector(delta)
            v = y + delta_vector
            yield v

# Автоморфизм кривой
def aut_sigma(xi,i,r):
    return xi * alpha^((r+1)*r^(i-1))

# Кодирование на второй ступени башни Гарсии-Штихтенота
def FGS_encode(N,l,m,sD,F,T2,f):
    n = m*N
    #print('Длина кода n =', n)
    r = F.characteristic()
    points_code = get_points_code(N,r,F)
    #print("Точки кода:", points_code)
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
#     print("L(l*𝑃∞):", L1)    
    k = len(L1)
    #print('Размерность кода k =', k)
    #print('Скорость кода R >=', round(float(k/n),5))
    d = N-l/m
    #print('Минимальное расстояние кода d >=', d)
    t_min = int((d-1)/2)
    g = T2.genus()
    kappa = ceil((N*(m-sD+1) - (k+g-1) + (sD+1)*(g/2-1) + 1) / (sD+1))
    t = (kappa+l) / (m-sD+1)
    t_max = ceil(N-t)
    #print('Число исправляемых ошибок:', t_min, '<= t <=', t_max)
    #print('Исходное сообщение f =', f)
    f_RR = 0
    for f_el, L1_el in zip(f, L1):
        f_RR += f_el*L1_el
    PR = PolynomialRing(F, 2, ['x1','x2'])
    f_RR = PR(f_RR)
    #print('Исходное сообщение как элемент L(l*𝑃∞):', f_RR)
    y = [[0 for i in range(0,N)] for j in range(0,m)]
    for i in range(0,m):
        for j in range(0,N):
            x_1 = points_code[j][0]
            x_2 = points_code[j][1]
            for count_sigma in range(0,i):
                x_1 = aut_sigma(x_1,1,r)
                x_2 = aut_sigma(x_2,2,r)
            y[i][j] = f_RR(x_1, x_2)
    return y

# Декодер кода на второй ступени башни Гарсии-Штихтенота
def FGS_decode(N,l,m,sD,F,T2,y):
    r = F.characteristic()
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
#     print('L(l*𝑃∞):', L1)
    k = len(L1)
    print("k =", k)
    g = T2.genus()
    D = ceil((N*(m-sD+1) - (k+g-1) + (sD+1)*(g/2-1) + 1) / (sD+1)) # Степенной параметр
    L2 = (D*P_inf).basis_function_space() # Базис пространства L(D*𝑃∞)
#     print('L(D*𝑃∞):', L2)
    L3 = ((l+D)*P_inf).basis_function_space() # Базис пространства L((l+D)*𝑃∞)
#     print('L((l+D)*𝑃∞):', L3)
    points_code = get_points_code(N,r,F)
    count_lines = N*(m-sD+1)
    count_columns = len(L3)+len(L2)
    A0_A1_matrix = matrix(F, count_lines, count_columns) # Матрица коэффициентов
    PR = PolynomialRing(F, 2, ['x1','x2'])
    for j in range(0,N):
        x_1 = points_code[j][0]
        x_2 = points_code[j][1]
        for count_sigma in range(0,m-sD+1):
            equation_coefficients = []
            for i in range(0,len(L3)):
                L3_i = PR(L3[i])
                equation_coefficients.append(L3_i(x_1, x_2))
            for i in range(0,len(L2)):
                L2_i = PR(L2[i])
                equation_coefficients.append(L2_i(x_1, x_2)*y[count_sigma][j])
            A0_A1_matrix[j*(m-sD+1)+count_sigma] = equation_coefficients
            x_1 = aut_sigma(x_1,1,r)
            x_2 = aut_sigma(x_2,2,r)
#     print("Матрица для нахождения A0 и A1:")
    #print(A0_A1_matrix)
    recovered_message_list = []
    for x in A0_A1_matrix.right_kernel().basis():
        #print("Частное решение:",x) 
        _.<Y> = PolynomialRing(F)
        A0_coeff = x[:len(L3)]
        A1_coeff = x[len(L3):]
        A0 = 0
        A1 = 0
        for coeff, L3_elem in zip(A0_coeff, L3):
            A0 += coeff*L3_elem
        for coeff, L2_elem in zip(A1_coeff, L2):
            A1 += coeff*L2_elem
        Q = A0 + A1*Y
    #    print("A0 =", A0)
    #    print("A1 =", A1)
    #    print("Q =", Q)

        count_lines = N
        count_columns = len(L1)
        f_matrix = matrix(F, count_lines, count_columns)
        b = [0]*count_lines
        A0_PR = PR(A0)
        A1_PR = PR(A1)
        for j in range(0,count_lines):
            x_1 = points_code[j][0]
            x_2 = points_code[j][1]
            equation_coefficients = []
            for i in range(0,count_columns):
                L1_i = PR(L1[i])
                equation_coefficients.append(L1_i(x_1, x_2)*A1_PR(x_1,x_2))
            f_matrix[j] = equation_coefficients
            b[j] = -A0_PR(x_1,x_2)
        b_vector = vector(b)
     #   print("Матрица для нахождения исходного сообщения:")
     #   for i in f_matrix:
     #       print(i)
     #   print("Столбец свободных членов:", b_vector)
        try:
            recovered_message = list(f_matrix.solve_right(b_vector))
            if (recovered_message not in recovered_message_list):
                recovered_message_list.append(recovered_message)
        except:
            print("Декодирование невозможно")
            pass
#         print("-------------------------")
    return recovered_message_list
    
# Декодирование решётки конструкции D
def LatticeDecoder(y,i_recursion_step,e0,l_divisor_multipliers,F,N,m,sD,T2):
    p = int(F.order())
    ei = (p^i_recursion_step)*e0
    w = [round(j) % p  for j in y]
    print("w =", w)
    if (i_recursion_step == 0):
        L = BaseFieldDecoder(w, e0)
#         print("BaseFieldDecoder", list(L))
    else:
        w = get_reverse_bijection_phi_vector(w, F, F.degree())
        print("w C"+str(i_recursion_step)+" =", w)
        l = l_divisor_multipliers[i_recursion_step-1]
        L = FGS_decode(N,l,m,sD,F,T2,[w])
        print("L C"+str(i_recursion_step)+": ", L)
        L = list(map(lambda word: FGS_encode(N,l,m,sD,F,T2,word)[0], L))
        L = list(map(lambda code_word: get_bijection_phi_vector(code_word, F.degree()), L))
    R_c = []
    for c in L:
        if i_recursion_step == 0:
#             print("y step 0 =", y)
#             print("c step 0 =", c)
            res = [b + (round((a-b) / p) * p) for a,b in zip(y, c)]
            R_c.append(res)
        else:
            print("y =", y)
            print("c =", c)
            y_new = [(a-b) / p for a,b in zip(y, c)]
            print("y_new =", y_new)
            for v in LatticeDecoder(y_new,i_recursion_step-1,e0,l_divisor_multipliers,F,N,m,sD,T2):
#                 print("v i"+str(i_recursion_step), v)
#                 print("c i"+str(i_recursion_step), c)
                rez = [a + p*b for a,b in zip(c,v)]
                if ((vector(y)-vector(rez)).norm() <= ei):
                    R_c.append(rez)
#     print("R_c i"+str(i_recursion_step), R_c)
    
    return R_c


i_recursion_step = 2 # значение i в Алгоритме 2 декодирования решётки Peikert
e0 = 1.5 # Граница нормы разности векторов ||v-y|| <= e0

L_len = L.nrows()
lattice_vector = 55*L[L_len-2]
error_vector = vector(ZZ, [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
y =  list(lattice_vector + error_vector)
print("y =", y)

l_divisor_multipliers = [l1, l2] # Множители дивизора l_i*𝑃∞ подкода C_i
vectors = LatticeDecoder(y,i_recursion_step,e0,l_divisor_multipliers,F,N,m,sD,T2)
print(vectors)
print(lattice_vector in vectors)

y = [0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 55, 55, 0, 0, 55, 55, 0, 0, 55, 55, 55, 0, 55, 1]
w = [0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 7, 7, 0, 0, 7, 7, 0, 0, 7, 7, 7, 0, 7, 1]
w C2 = [alpha^3, alpha^3, alpha^3, alpha^3, alpha^3, alpha^3, alpha^3, alpha^3, alpha^3 + alpha^2, alpha^3 + alpha^2, alpha^3 + alpha^2, alpha^3 + alpha^2 + 1]
k = 2
L C2:  [[1, 0]]
y = [0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 0, 55, 0, 0, 55, 55, 0, 0, 55, 55, 0, 0, 55, 55, 55, 0, 55, 1]
c = [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1]
y_new = [0, 0, 0, 27/8, 0, 0, 0, 27/8, 0, 0, 0, 27/8, 0, 0, 0, 27/8, 0, 0, 0, 27/8, 0, 0, 0, 27/8, 0, 0, 0, 27/8, 0, 0, 0, 27/8, 0, 0, 27/8, 27/8, 0, 0, 27/8, 27/8, 0, 0, 27/8, 27/8, 55/16, 